# Human Activity Recognition

### Step 1: Download the dataset from link 

https://www.utwente.nl/en/eemcs/ps/dataset-folder/sensors-activity-recognition-dataset-shoaib.rar


In [ ]:
import urllib.request

print('Downloading dataset')

url = 'https://www.utwente.nl/en/eemcs/ps/dataset-folder/sensors-activity-recognition-dataset-shoaib.rar'
urllib.request.urlretrieve(url, 'HAR_dataset.rar')

print('Download completed')

### Step 2: Extract in the working directory


In [ ]:
!pip3 install pyunpack
!pip3 install patool
from pyunpack import Archive
Archive('HAR_dataset.rar').extractall('.')

### Step 3: Download Libraries


In [ ]:
!pip3 install tensorflow==1.13.1
!pip3 install tensorflow-gpu==1.13.1
!pip3 install keras==2.2.0
!pip3 install pandas
!pip3 install numpy==1.14

In [ ]:
# import libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
path='dataset/'
files=[file for file in os.listdir(path) if file.endswith('.csv')]
files

In [ ]:
# load a file into memory
df=pd.read_csv(path+files[0],header=1)
df.head()

In [ ]:
df['Unnamed: 69'].unique()

## Visualize the difference in change in values w.r.t activity

In [ ]:
# change in values while walking

plt.plot(np.arange(0,100),df.Ax[df['Unnamed: 69']=="walking"][:100], label='X-axis')
plt.plot(np.arange(0,100),df.Ay[df['Unnamed: 69']=="walking"][:100], label='Y-axis')
plt.plot(np.arange(0,100),df.Az[df['Unnamed: 69']=="walking"][:100], label='Z-axis')
plt.legend()

In [ ]:
# change in values while standing

plt.plot(np.arange(0,100),df.Ax[df['Unnamed: 69']=="standing"][:100], label='X-axis')
plt.plot(np.arange(0,100),df.Ay[df['Unnamed: 69']=="standing"][:100], label='Y-axis')
plt.plot(np.arange(0,100),df.Az[df['Unnamed: 69']=="standing"][:100], label='Z-axis')
plt.legend()

In [ ]:
# change in values while biking

plt.plot(np.arange(0,100),df.Ax[df['Unnamed: 69']=="biking"][:100], label='X-axis')
plt.plot(np.arange(0,100),df.Ay[df['Unnamed: 69']=="biking"][:100], label='Y-axis')
plt.plot(np.arange(0,100),df.Az[df['Unnamed: 69']=="biking"][:100], label='Z-axis')
plt.legend()

In [ ]:
# change in values while jogging

plt.plot(np.arange(0,100),df.Ax[df['Unnamed: 69']=="jogging"][:100], label='X-axis')
plt.plot(np.arange(0,100),df.Ay[df['Unnamed: 69']=="jogging"][:100], label='Y-axis')
plt.plot(np.arange(0,100),df.Az[df['Unnamed: 69']=="jogging"][:100], label='Z-axis')
plt.legend()

## Load and concatenate all files into one dataframe

In [ ]:
df=pd.DataFrame()

for file in files:
    df_temp=pd.read_csv(path+file,header=1)
    df=pd.concat([df,df_temp], sort=False)
df

In [ ]:
for i in df.columns:
    print(i)

In [ ]:
# split out left and right pocket data

left_pocket = df[df.columns[1:10]]
left_pocket

In [ ]:
right_pocket = df[df.columns[15:24]]
right_pocket.columns=left_pocket.columns
right_pocket

In [ ]:
# concatenate left and right split in one data frame

train_df=pd.concat([left_pocket,right_pocket],sort=False)
train_df

In [ ]:
# restore labels in dataframe
labels=pd.concat([df['Unnamed: 69'],df['Unnamed: 69']],axis=0, sort=False)
labels.columns=['Activity']
train_df['Activity']=labels
train_df

In [ ]:
#check for unique labels
train_df.Activity.unique()

In [ ]:
train_df.Activity.loc[(train_df.Activity == 'upsatirs')] = 'upstairs'
train_df.Activity.unique()

# Preprocessing 


## Split the data and labels

In [ ]:
X_train = train_df[train_df.columns[:9]]
y_train = train_df[train_df.columns[9:10]]

In [ ]:
X_train 

In [ ]:
y_train

In [ ]:
y_train.Activity.unique()

## Encode labels into number

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()
y_train=encoder.fit_transform(y_train)
y_train[:10]

## Split training data into training and validation data

In [ ]:
def train_test_split(X, y, split_size=0.8):
    split= int(len(X) * split_size)
    train_x = X[:split]
    train_y = y[:split]
    test_x = X[split:]
    test_y = y[split: ]
    return train_x, test_x, train_y, test_y

X_train,X_test,y_train, y_test =train_test_split(X_train, y_train)

print("X_train shape ", X_train.shape)
print("Y train shape ", y_train.shape)
print("X_test shape ", X_test.shape)
print("y_test shape ", y_test.shape)

## Convert dataset into time series sequence

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator

n_time_steps = 100
n_features = 9

train_gen = TimeseriesGenerator(X_train.to_numpy(), y_train, length=n_time_steps, batch_size=1024)
test_gen = TimeseriesGenerator(X_test.to_numpy(), y_test, length=n_time_steps, batch_size=1024)

## Create and compile LSTM model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Flatten, LSTM
from keras.regularizers import l2
from keras.optimizers import Adam

In [ ]:
model=Sequential()
model.add(LSTM(32, return_sequences=True, input_shape = (n_time_steps, n_features),
              kernel_regularizer = l2(0.000001), bias_regularizer = l2(0.000001), name='lstm_1'))
model.add(Flatten(name='flatten'))
model.add(Dense(64, activation='relu',kernel_regularizer = l2(0.000001), bias_regularizer = l2(0.000001), name='dense_1' ))
model.add(Dense(len(np.unique(y_train)), activation='softmax', 
                kernel_regularizer = l2(0.000001), bias_regularizer = l2(0.000001), name='output'))
model.summary()

In [ ]:
#compile
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
# prepare callbacks
from keras.callbacks import ModelCheckpoint

callbacks= [ModelCheckpoint('model.h5', save_weights_only=False, save_best_only=True, verbose=1)]

## Start training

In [ ]:
history = model.fit_generator(train_gen, epochs=5, validation_data=test_gen, callbacks=callbacks)

In [ ]:
# stopping training here

# loading the best saved model 

from keras.models import load_model
model=load_model('model.h5')
model.summary()

# Exporting the model

In [ ]:
from keras import backend as k
from tensorflow.python.tools import freeze_graph, optimize_for_inference_lib
import tensorflow as tf

input_node_name = ['lstm_1_input']
output_node_name = 'output/Softmax'
model_name='har_model'

In [ ]:
tf.train.write_graph(k.get_session().graph_def, 'models', model_name + '_graph.pbtxt')
saver=tf.train.Saver()
saver.save(k.get_session(), 'models/'+model_name + '.chkp')

In [ ]:
freeze_graph.freeze_graph('models/'+model_name + '_graph.pbtxt', None, False, 'models/'+model_name+'.chkp',
                         output_node_name, 'save/restore_all', 'save/Const:0', 'models/frozen_' + model_name + '.pb', True, "")

In [ ]:
# Deployment of this model in next video 
